In [1]:
!pip install mlflow boto3 awscli optuna xgboost imbalanced-learn

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/150.0 MB ? eta -:--:--
   ---------------------------------------- 0.5/150.0 MB 210.0 kB/s eta 0:11:52
   ---------------------------------------- 0.5/150.0 MB 210.0 kB/s eta 0:11:52
   ---------------------------------------- 0.5/150.0 MB 210.0 kB/s eta 0:11:52
   ---------------------------------------- 0.5/15

In [2]:
import mlflow
mlflow.set_tracking_uri("http://ec2-18-215-180-118.compute-1.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

2025/07/15 23:44:32 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-1218/950194470795105794', creation_time=1752601474061, experiment_id='950194470795105794', last_update_time=1752601474061, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [5]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 4.8 MB/s eta 0:00:01
   --------------------- ------------------ 0.8/1.5 MB 2.0 MB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 1.5 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 1.4 MB/s eta 0:00:00


In [6]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [8]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [9]:
import os
os.environ["MLFLOW_HTTP_REQUEST_TIMEOUT"] = "300"
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Step 4: Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()

[I 2025-07-15 23:50:52,488] A new study created in memory with name: no-name-e1c27f3a-2266-433a-b8c2-288ba143663f
[I 2025-07-15 23:55:46,304] Trial 0 finished with value: 0.6128460384562935 and parameters: {'n_estimators': 113, 'learning_rate': 0.0020730346547379107, 'max_depth': 9}. Best is trial 0 with value: 0.6128460384562935.
[I 2025-07-16 00:07:45,373] Trial 1 finished with value: 0.6729851356879858 and parameters: {'n_estimators': 229, 'learning_rate': 0.007392353368860532, 'max_depth': 10}. Best is trial 1 with value: 0.6729851356879858.
[I 2025-07-16 00:08:42,176] Trial 2 finished with value: 0.6529387699440884 and parameters: {'n_estimators': 189, 'learning_rate': 0.023627728318642332, 'max_depth': 3}. Best is trial 1 with value: 0.6729851356879858.
[I 2025-07-16 00:10:06,162] Trial 3 finished with value: 0.7017591708714033 and parameters: {'n_estimators': 104, 'learning_rate': 0.06202583370229551, 'max_depth': 5}. Best is trial 3 with value: 0.7017591708714033.
[I 2025-07-16

KeyboardInterrupt: 